# The distribution of Encounters

This notebook shows how to infer shock-histories, by properly Poisson-sampling the corresponding distribution

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../adiabatic-tides")
sys.path.append("..")

import adiabatic_tides as at
import cusp_encounters.milkyway
import cusp_encounters.encounters_math as em
import scipy.constants as const

cachedir = "../caches"

G = 43.0071057317063e-10 # Mpc (km/s)^2 / Msol 

# Spatial distribution of encounters (not relevant)

This is a heuristic estimate of the spatial distribution of encounters. This is not relevant, but still interesting

$$r_0 = \frac{1}{n_*}$$

is the mean distance to an object in the projected encounter plane, where $n_*$ is the number column density.

E.g. if all stars had a mass of $1 M_\odot$ then $n_* = \Sigma_* / M_\odot$ where $\Sigma_*$ is the stellar mass column density.

In [ ]:
ri = np.logspace(-2,1, 500)
plt.plot(ri, em.knn_distribution(ri, 1., k=1) * ri, label="closest", lw=2)
plt.plot(ri, em.knn_distribution(ri, 1., k=2) * ri, label="2nd-closest", lw=2)
plt.plot(ri, em.knn_distribution(ri, 1., k=3) * ri, label="3rd-closest", lw=2)

plt.grid()
plt.xlabel(r"$b / r_0$", fontsize=14)
plt.xscale("log")
plt.ylabel(r"d$N$ / d ln $(r/r_0)$", fontsize=14)
plt.xlim(3e-2, 2e0)
plt.legend(fontsize=14)

plt.savefig("img/closest_encounter_distribution.pdf", bbox_inches="tight")

For example, if we had $n_* = 10^4 / \rm{pc}^2$ (typical for cusps at 10 kpc), then we have a median closest encounter of about $0.5 \times 10^{-2} \rm{pc}$

# Distribution of shock parameters

Given some value of $B_*$ what is the distribution of the effective shock parameter $B_{\rm{eff}} = \sqrt[1.2]{\sum B_i^{1.2}}$ ? How does it depend on the number of shocks that are considered?

In [ ]:
%%time
Nsamp = 100000 # reduce this e.g. to 10000 if you don't want to wait very long or you have low memory available
Bmin = 1e-2
Bstar = 1.
B = em.sample_encounters_B(Nsamp, Bmin, Bstar=Bstar)

Beff1 = em.sample_effective_B(Nsamp, Bstar=1., Bmin=1e-1, p=1.2)
Beff2 = em.sample_effective_B(Nsamp, Bstar=1., Bmin=1e-2, p=1.2)
Beff3 = em.sample_effective_B(Nsamp, Bstar=1., Bmin=1e-3, p=1.2)
Beff4 = em.sample_effective_B(Nsamp, Bstar=1., Bmin=1e-4, p=1.2)

print(np.median(Beff4))

In [ ]:
%%time
# This is an accelerator function, that samples from a first calculated histogram which is also be cached
# Later on we use this to accelerate computations
# On first execution ~ 3 minutes, but later ~50ms
Beff4_quick= em.sample_effective_B_hist(100000, Bminfac=1e-4, p=1.2, initial_sample=100000, cachefile="%s/Beff_hist.hdf5" % cachedir)

In [ ]:
bins = np.logspace(-1.5,3.5,100)
dbin = np.log(bins[1:]) - np.log(bins[:-1])
Bcent = np.sqrt(bins[1:]*bins[:-1])

ni,_ = np.histogram(B[:,0], bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label="1st strongest", lw=2)

ni,_ = np.histogram(B[:,1], bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label="2nd strongest", lw=2)

ni,_ = np.histogram(B[:,2], bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label="3rd strongest", lw=2)

ni,_ = np.histogram(Beff1, bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label=r"$B_{\rm{eff}}$ [$B > 10^{-1} B_*$]", lw=2,  ls="dashed") #color=plt.get_cmap("viridis")(0.),

ni,_ = np.histogram(Beff2, bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label=r"$B_{\rm{eff}}$ [$B > 10^{-2} B_*$]", lw=2,  ls="dashed") #color=plt.get_cmap("viridis")(0.3),

ni,_ = np.histogram(Beff3, bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label=r"$B_{\rm{eff}}$ [$B > 10^{-3} B_*$]", lw=2, ls="dashed")

ni,_ = np.histogram(Beff4, bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label=r"$B_{\rm{eff}}$ [$B > 10^{-4} B_*$]", lw=2, ls="dashed")

plt.xscale("log")

def fana(B, Bstar):
    return Bstar/B**2 * np.exp(-Bstar/B)
    
plt.xlabel(r"$B / B_*$", fontsize=12)
plt.ylabel(r"dN / d log B", fontsize=12)
plt.grid("grid")
plt.legend(fontsize=12)

plt.xlim(5e-2,4e3)

plt.savefig("img/shock_distribution.pdf", bbox_inches="tight")

# Two small checks that we don't put in the publication figure
plt.plot(Bcent, fana(Bcent, Bstar)*Bcent, color="black", ls="dotted", label="analytic distribution", lw=2)
ni,_ = np.histogram(Beff4_quick, bins=bins)
plt.plot(Bcent, ni/dbin/Nsamp, label=r"quick [$B > 10^{-4} B_*$]", lw=2, ls="dashed", color="black")

# Encounters in the Milky Way

In [ ]:
#mw = cusp_encounters.milkyway.MilkyWay(adiabatic_contraction=True, cachedir=cachedir, mode="cautun_2020")
#orbits_full = mw.create_dm_orbits(100000, nsteps=100000, rmax=500e3, addinfo=True, adaptive=True, subsamp=100)

mw = cusp_encounters.milkyway.MilkyWay(adiabatic_contraction=True, cachedir=cachedir, mode="cautun_2020")
import create_orbits_mpi
orbits = create_orbits_mpi.orbits

In [ ]:
# Sample down a bit more to keep things quick
#subsamp = 100
#orbits = {} 
#for key in "chi_star", "pos", "chi_dm":
#    orbits[key] = orbits_full[key][::subsamp]
#orbits["mass"] = orbits_full["mass"]

In [ ]:
subsamp = 1

r = np.linalg.norm(orbits["pos"][::subsamp], axis=-1)
chi = orbits["chi_star"][-1] * np.ones_like(orbits["chi_star"][::subsamp])
chi = np.clip(chi, 1e-8, None)
chitot = (orbits["chi_star"][-1] + 0.04*orbits["chi_dm"][-1]) * np.ones_like(orbits["chi_star"][::subsamp])
mass = orbits["mass"]*np.ones_like(orbits["chi_star"][::subsamp])

Bstrongest = em.sample_strongest_B_analytic(2.*np.pi*chi*mw.G)
ri, perc = em.get_percentile_profile(r/1000., Bstrongest, weights=mass)

Bstrongest_tot = em.sample_strongest_B_analytic(2.*np.pi*chitot*mw.G)
ri, perc2 = em.get_percentile_profile(r/1000., Bstrongest_tot, weights=mass)

In [ ]:
plt.axhline(0.3, ls="--", label=r"$B_{\rm{cusp}}, B_{\rm{core}}$", color="red", lw=2)
plt.axhline(30., ls="--", color="red", lw=2)
em.percentile_plot(ri, perc, ylabel=r"Strongest Shock $B$ [km/s / pc]", loc="lower left")
plt.ylim(1e-4, 5e4)

def B_to_b(B):
    Mstar, v = 1., 200
    return np.sqrt(2.*Mstar*mw.G / (B * v)) * (const.parsec/const.au)
def b_to_B(b):
    Mstar, v = 1., 200
    return 2.*Mstar*mw.G / (b**2 * v) / (const.parsec/const.au)**2
ax_chiG = plt.gca().secondary_yaxis('right', functions=(B_to_b, b_to_B))
ax_chiG.set_ylabel("closest encounter $b$ [AU]", fontsize=14)

plt.legend(loc="upper right", fontsize=12)
plt.savefig("img/radial_strongest_shock_and_encounter.pdf", bbox_inches="tight")

# Effective Shock

In [ ]:
Beff = em.sample_effective_B_hist(chi.shape, Bminfac=1e-4, p=1.2, initial_sample=100000, cachefile="%s/Beff_hist.hdf5" % cachedir) * (2.*np.pi*chi*mw.G)
ri, perc = em.get_percentile_profile(r/1000., Beff, weights=mass)

Beff2 = em.sample_effective_B_hist(chi.shape, Bminfac=1e-4, p=1.2, initial_sample=100000, cachefile="%s/Beff_hist.hdf5" % cachedir) * (2.*np.pi*chitot*mw.G)
ri, perc2 = em.get_percentile_profile(r/1000., Beff2, weights=mass)

In [ ]:
plt.axhline(0.224, ls="--", label=r"$B_{\rm{cusp}}, B_{\rm{core}}$", color="red", lw=2)
plt.axhline(23.7, ls="--", color="red", lw=2)
em.percentile_plot(ri, perc, ylabel=r"Effective Shock $B_{\rm{eff}}$ [km/s / pc]", loc="lower left")
plt.ylim(1e-4, 5e4)

plt.legend(loc="upper right", fontsize=12)
plt.savefig("img/radial_effective_shock.pdf", bbox_inches="tight")